# agent with Llamaindex


# Agent with LlamaIndex Notebook

This notebook demonstrates the implementation of a simple agent using LlamaIndex framework. It showcases:

- Setting up a local LLM model using Ollama
- Creating custom functions for multiplication and weather data retrieval
- Implementing an agent workflow with function tools
- Demonstrating both stateless and stateful agent interactions
- Using context objects to maintain conversation history

The agent is able to perform mathematical operations and fetch weather information through defined function tools.


In [ ]:
pip install llama-index llama-index-llms-ollama

In [ ]:
pip install --upgrade typing_extensions # use python version >= 3.10 or update type extensions

In [ ]:
from llama_index.core.agent.workflow import (AgentWorkflow,
                                             FunctionAgent,
                                             ReActAgent)
from llama_index.core.tools import FunctionTool
from llama_index.llms.ollama import Ollama
# remembering state
from llama_index.core.workflow import Context


# initialize local model

In [ ]:
llm = Ollama(model="dagbs/qwen2.5-coder-3b-instruct-abliterated:latest")
knowledge_llm = Ollama(model="llama3.2:1b",
                       request_timeout=120.0,
                        context_window=2048,
                       additional_kwargs={
                           "num_gpu":1,
                           "num_thread":4,
                           "nlock":False,
                           "numa":False
                       })

## functions

In [ ]:
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

def multiply(a:int, b:int) -> int:
    """Multiply two numbers and return the result."""
    return a * b

In [ ]:
import requests

def get_weather(city: str) -> str:
    """ a function that returns the weather data as a string value"""
    url = "https://api.weatherapi.com/v1/current.json"
    params = {"key": "e40acd9bb9154ea499d34832252505", "q": city, "aqi":"no"}

    resp = requests.get(url, params=params)
    if resp.status_code == 200:
        response_data = resp.json()
        return response_data["current"]["condition"]["text"]
    else:
        raise Exception("not data found")
    # return f"The weather in {city} is sunny today."

In [ ]:
# agent configurations

calculator_agent = ReActAgent(
    name = "Calculator",
    description="Performs basic arithmetic operations like addition, subtraction, multiplication.",
    system_prompt="you are a calculator assistant, Use your tools for any math operations",
    tools=[add, subtract, multiply],
    llm=llm
)

weather_agent = ReActAgent(
    name="weather",
    description="Useful for getting the weather for a given location.",
    system_prompt="you are a weather assistant, Use your tools for any weather information",
    tools=[get_weather],
    llm=llm
)

knowledge_agent = ReActAgent(
    name="orchestrator",
    description="Useful for to think and remember contexts. and act as an orchestrator agent.",
    system_prompt="you are an orchestrator agent, answer questions based on the context. and assign the task to the right agent if necessarily. make sure your answer is concise, accurate, short and the result not the process. example if anybody ask about weather in specific city use the tool the weather tool provided, if someone ask basic math use calculator tool. rather than that use knowledge to memorize and understand the context.",
    llm=knowledge_llm
)



## agent

In [ ]:
agent = AgentWorkflow(agents=[calculator_agent, weather_agent, knowledge_agent], root_agent="orchestrator")

In [ ]:
# initialize context
ctx = Context(agent)

Agents are stateless by default, add remembering past interactions is opt-in using a Context object This might be useful if you want to use an agent that needs to remember previous interactions, like a chatbot that maintains context across multiple messages or a task manager that needs to track progress over time.

In [ ]:
# stateless
response = await agent.run(user_msg="What is 2 times 3?")
print(response)

In [ ]:
print(await agent.run(user_msg="my name is john", ctx=ctx))

In [ ]:
print(await agent.run(user_msg="what is my name?", ctx=ctx))

In [ ]:
print(await agent.run(user_msg="what is the weather in washington DC?"))

In [ ]:
print(await agent.run(user_msg="create a python function that sort a list of numbers by using bubble sort algorithm"))